In [ ]:
# %pip install numpy scipy pandas plotly nbformat


In [ ]:
import numpy as np
from numpy.fft import fft, ifft
import scipy
import scipy.signal
import plotly.express as px
import plotly.graph_objects as go

from circular_convolution import cc_direct, cc_fft


In [ ]:
N = 128
x = np.linspace(0, 50, N)

true_y = np.sin(x)
y_range = np.max(true_y) - np.min(true_y)
y = true_y + np.random.normal(scale=y_range/10, size=N) + np.sin(x*1.33)

fig = px.line(y=y)
# fig.add_scatter(y=scipy.signal.savgol_filter(y, 20, 5), name='smooth')
fig.show()


In [ ]:
kernel = np.concat([np.zeros((N//4, )), np.ones((N//2,)), np.zeros((N//4,))])
# kernel = np.random.normal(scale=1, size=N)

y_direct = cc_direct(y, kernel)
y_fft = cc_fft(y, kernel)
fig = go.Figure()
fig.add_scatter(y=y_direct)
fig.add_scatter(y=y_fft)
fig.show()

print('Standard deviation:', np.var(y_direct - y_fft))
